In [1]:
import os
import sys

os.chdir('..')
sys.path.append(os.getcwd())

In [2]:
from config import JSONConfig

# load config
cfg = JSONConfig('/mnt/c/ti/mmwave_studio_03_00_00_07/mmWaveStudio/PostProc/hiber2_408/hiber2_408.mmwave.json')

In [3]:
import matplotlib.pyplot as plt
from radardsp.antennas import load_virtual_array_layout

rxl, txl = load_virtual_array_layout('./antenna_cfg.txt')
######### plot virtual array layout #########
x = []
y = []
va_pos = []
for tidx, taz, tel in txl:
    for ridx, raz, rel in rxl:
        x.append(taz+raz) # RX_id_tot in MATLAB
        y.append(tel+rel) # RX_id_tot_ele in MATLAB
        va_pos.append((taz+raz, tel+rel))
# remove duplicates
va_pos = list(set(va_pos))

# plot the virtual antenna array layout
for _, __ in va_pos:
        plt.plot(_, __, 'o')
#############################################

AssertionError: Layout file does not exist